# CLI Commands & Patterns

**Master dynamic content inclusion with commands and file patterns.**

## What You'll Learn

- Execute CLI commands and include their output
- Use glob patterns to include multiple files
- Apply the `:expand:` modifier for recursive processing
- Combine techniques for powerful workflows
- Build practical documentation automation

## Prerequisites

- Complete Notebook 00 (Getting Started)
- Basic command line knowledge
- Understanding of file paths and glob patterns

**Duration**: ~20 minutes  
**Level**: Intermediate

## Setup

Let's create a sample project to work with.

In [ ]:
# Install gtext
!pip install -q gtext

# Create demo project
import os
from pathlib import Path

os.makedirs('demo_cli/docs', exist_ok=True)
os.makedirs('demo_cli/src', exist_ok=True)
os.makedirs('demo_cli/tests', exist_ok=True)
os.makedirs('demo_cli/templates', exist_ok=True)

print("✅ Demo project created!")
print("📁 demo_cli/")
print("  ├── docs/       (documentation templates)")
print("  ├── src/        (source code)")
print("  ├── tests/      (test files)")
print("  └── templates/  (reusable templates)")

## Example 1: CLI Commands - Dynamic Content

In Notebook 00, you included static files. Now let's include **command output**.

### Use Case: System Status Report

In [ ]:
%%writefile demo_cli/docs/status-report.md.gtext
# System Status Report

## Current Date

```include
cli: date
```

## Disk Usage

```include
cli: df -h | head -5
```

## Current User

```include
cli: whoami
```

In [ ]:
# Generate the report
!cd demo_cli && gtext render docs/status-report.md.gtext

# View result
print("\n📄 Generated Report:\n")
with open('demo_cli/docs/status-report.md', 'r') as f:
    print(f.read())

**Key Points:**
- `cli:` protocol executes shell commands
- Commands run in the template's directory
- Output is captured and included in the document
- Perfect for live data that changes frequently

## Example 2: Glob Patterns - Multiple Files

Include multiple files matching a pattern.

### Setup: Create Sample Code Files

In [ ]:
%%writefile demo_cli/src/module_a.py
"""Module A - User authentication."""

def login(username: str, password: str) -> bool:
    """Authenticate user."""
    # Implementation here
    return True

In [ ]:
%%writefile demo_cli/src/module_b.py
"""Module B - Data processing."""

def process_data(data: list) -> dict:
    """Process input data."""
    # Implementation here
    return {}

In [ ]:
%%writefile demo_cli/src/module_c.py
"""Module C - Reporting."""

def generate_report(data: dict) -> str:
    """Generate report from data."""
    # Implementation here
    return "Report"

### Use Case: Code Listing Document

In [ ]:
%%writefile demo_cli/docs/code-listing.md.gtext
# Source Code Listing

## All Python Modules

```include
glob: src/*.py
```

---
*Auto-generated from source files*

In [ ]:
# Generate code listing
!cd demo_cli && gtext render docs/code-listing.md.gtext

# View result
print("\n📄 Code Listing:\n")
with open('demo_cli/docs/code-listing.md', 'r') as f:
    print(f.read())

**Key Points:**
- `glob:` matches multiple files with patterns
- Standard glob syntax: `*` (any), `**` (recursive), `?` (single char)
- Files are included in sorted order
- Each file is separated by a blank line

## Example 3: Combining CLI and Glob

Mix both techniques for richer documents.

### Use Case: Project Overview

In [ ]:
%%writefile demo_cli/docs/project-overview.md.gtext
# Project Overview

**Generated:** 
```include
cli: date
```

## File Structure

```include
cli: tree -L 2 -I '__pycache__|*.pyc'
```

## Source Files

**Total Python files:**
```include
cli: find src -name '*.py' | wc -l
```

**Lines of code:**
```include
cli: find src -name '*.py' -exec wc -l {} + | tail -1
```

## Module Listing

```include
glob: src/*.py
```

In [ ]:
# Generate overview
!cd demo_cli && gtext render docs/project-overview.md.gtext

# View result (first 50 lines)
print("\n📄 Project Overview:\n")
with open('demo_cli/docs/project-overview.md', 'r') as f:
    lines = f.readlines()
    print(''.join(lines[:50]))
    if len(lines) > 50:
        print(f"\n... ({len(lines) - 50} more lines)")

## Example 4: The `:expand:` Modifier

The `:expand:` modifier tells gtext to **recursively process** included files.

### Problem Without Expand

In [ ]:
%%writefile demo_cli/templates/git-info.md.gtext
## Git Information

**Branch:**
```include
cli: git branch --show-current
```

**Recent commits:**
```include
cli: git log --oneline -5
```

In [ ]:
%%writefile demo_cli/docs/report-no-expand.md.gtext
# Development Report

```include
static: templates/git-info.md.gtext
```

## Analysis

More content here...

In [ ]:
# Initialize git repo for demo
!cd demo_cli && git init 2>/dev/null || true
!cd demo_cli && git config user.email "demo@example.com" 2>/dev/null || true
!cd demo_cli && git config user.name "Demo User" 2>/dev/null || true
!cd demo_cli && git add . 2>/dev/null || true
!cd demo_cli && git commit -m "Initial commit" 2>/dev/null || true

# Render without expand
!cd demo_cli && gtext render docs/report-no-expand.md.gtext

print("\n⚠️ Result WITHOUT :expand:\n")
with open('demo_cli/docs/report-no-expand.md', 'r') as f:
    print(f.read())

**Notice:** The ```include blocks from git-info.md.gtext are NOT processed - they appear as-is!

### Solution: Use `:expand:`

In [ ]:
%%writefile demo_cli/docs/report-with-expand.md.gtext
# Development Report

```include
:expand:static: templates/git-info.md.gtext
```

## Analysis

More content here...

In [ ]:
# Render WITH expand
!cd demo_cli && gtext render docs/report-with-expand.md.gtext

print("\n✅ Result WITH :expand:\n")
with open('demo_cli/docs/report-with-expand.md', 'r') as f:
    print(f.read())

**Key Points:**
- Without `:expand:` - Included file is treated as static text
- With `:expand:` - gtext processes ```include blocks in the included file
- Enables composable templates that can include other templates
- Can nest multiple levels (template includes template includes...)

## Example 5: Practical Use Case - Automated Changelog

Let's build something real: an auto-generated changelog.

In [ ]:
# Create some git history
!cd demo_cli && echo "# Test" > test1.txt && git add test1.txt && git commit -m "feat: Add user login" 2>/dev/null
!cd demo_cli && echo "# Test2" > test2.txt && git add test2.txt && git commit -m "fix: Resolve null pointer error" 2>/dev/null
!cd demo_cli && echo "# Test3" > test3.txt && git add test3.txt && git commit -m "docs: Update README" 2>/dev/null
!cd demo_cli && echo "# Test4" > test4.txt && git add test4.txt && git commit -m "feat: Add data export" 2>/dev/null

print("✅ Created git history with 5 commits")

In [ ]:
%%writefile demo_cli/docs/CHANGELOG.md.gtext
# Changelog

All notable changes to this project are documented here.

**Generated:** 
```include
cli: date
```

## Recent Changes

```include
cli: git log --pretty=format:"- **%s** (%h) - %ar" -10
```

## Contributors

```include
cli: git log --format='%an' | sort -u
```

## Statistics

**Total commits:**
```include
cli: git rev-list --count HEAD
```

**First commit:**
```include
cli: git log --reverse --format="%h - %s" | head -1
```

---
*This changelog is automatically generated using gtext*

In [ ]:
# Generate changelog
!cd demo_cli && gtext render docs/CHANGELOG.md.gtext

print("\n📄 Generated CHANGELOG:\n")
with open('demo_cli/docs/CHANGELOG.md', 'r') as f:
    print(f.read())

**Benefits:**
- Always up-to-date with latest commits
- No manual changelog maintenance
- Can regenerate anytime with `gtext refresh`
- Customizable format with different git commands

## Example 6: Advanced Patterns

### Recursive Glob

Include files from subdirectories:

In [ ]:
# Create nested structure
!mkdir -p demo_cli/src/utils
!echo '# Utility A' > demo_cli/src/utils/util_a.py
!echo '# Utility B' > demo_cli/src/utils/util_b.py

In [ ]:
%%writefile demo_cli/docs/all-code.md.gtext
# All Source Code

## Recursive listing of all Python files:

```include
glob: src/**/*.py
```

In [ ]:
!cd demo_cli && gtext render docs/all-code.md.gtext

print("\n📄 Recursive Glob Result:\n")
with open('demo_cli/docs/all-code.md', 'r') as f:
    print(f.read()[:500] + "\n...")

### Multiple Patterns

Combine multiple includes:

In [ ]:
%%writefile demo_cli/docs/multi-pattern.md.gtext
# Code and Tests

## Source Modules

```include
glob: src/*.py
```

## Test Modules

```include
glob: tests/*.py
```

## Configuration Files

```include
glob: *.json
glob: *.toml
glob: *.yaml
```

## Example 7: Debugging with --dry-run

Preview what commands would execute without running them.

In [ ]:
%%writefile demo_cli/docs/debug-example.md.gtext
# Debug Example

```include
cli: echo "This would run"
```

```include
glob: src/*.py
```

In [ ]:
# Preview without executing
!cd demo_cli && gtext render docs/debug-example.md.gtext --dry-run

**Use --dry-run to:**
- Preview output before writing files
- Debug template syntax
- Verify glob patterns match expected files
- Test CLI commands safely

## Protocol Handlers Summary

| Protocol | Purpose | Example |
|----------|---------|-------|
| `static:` | Include file content (default) | `static: header.md` |
| `cli:` | Execute command, include output | `cli: git status` |
| `glob:` | Include files matching pattern | `glob: src/**/*.py` |

### Modifiers

| Modifier | Effect | Example |
|----------|--------|-------|
| `:expand:` | Recursively process included file | `:expand:static: template.gtext` |

Can combine: `:expand:cli:`, `:expand:glob:`, etc.

## Best Practices

### 1. Keep CLI Commands Simple

✅ **Good:**
```markdown
```include
cli: git log --oneline -5
```
```

❌ **Avoid:**
```markdown
```include
cli: git log --all --format='%h %s' | grep -v 'WIP' | awk '{print $1}' | xargs -I {} git show {}
```
```

Instead, create a shell script and call it:
```markdown
```include
cli: ./scripts/generate-changelog.sh
```
```

### 2. Use Descriptive Template Names

✅ `status-report.md.gtext`  
✅ `api-documentation.md.gtext`  
✅ `weekly-metrics.md.gtext`

❌ `template1.gtext`  
❌ `doc.gtext`  
❌ `test.md.gtext`

### 3. Create Reusable Template Components

```
templates/
├── git-info.md.gtext       # Reusable git information
├── system-info.md.gtext    # System stats
├── code-metrics.md.gtext   # Code statistics
└── header.md.gtext         # Common header
```

Include with `:expand:` in main documents.

### 4. Document Your Commands

Add comments to explain complex patterns:

```markdown
## Recent Contributors

<!-- Get unique authors from last 30 days -->
```include
cli: git log --since='30 days ago' --format='%an' | sort -u
```
```

### 5. Test Templates Incrementally

1. Start with static includes
2. Add one CLI command
3. Test with `--dry-run`
4. Render and verify
5. Add more complexity

## Real-World Examples

### Example: Pull Request Template

```markdown
# Pull Request: [Title]

## Changes

```include
cli: git diff main...HEAD --stat
```

## Modified Files

```include
cli: git diff main...HEAD --name-only
```

## Commits

```include
cli: git log main..HEAD --pretty=format:"- %s (%h)"
```

## Testing

- [ ] Unit tests pass
- [ ] Integration tests pass
- [ ] Manual testing completed
```

### Example: API Documentation

```markdown
# API Reference

**Last updated:** 
```include
cli: date
```

## Endpoints

```include
cli: grep -r '@app.route' src/ | sed 's/.*@app.route/Route:/'
```

## Models

```include
glob: src/models/*.py
```
```

### Example: Test Coverage Report

```markdown
# Test Coverage Report

**Generated:** 
```include
cli: date
```

## Coverage Summary

```include
cli: pytest --cov=src --cov-report=term
```

## Test Files

```include
glob: tests/**/*_test.py
```
```

## Summary

Congratulations! You've mastered dynamic content inclusion:

✅ **CLI Commands** - Execute commands and include output

✅ **Glob Patterns** - Include multiple files with patterns

✅ **Expand Modifier** - Recursively process templates

✅ **Combined Techniques** - Mix protocols for powerful workflows

✅ **Real Examples** - Changelogs, reports, documentation

✅ **Best Practices** - Reusable components, testing, organization

## Command Reference

```bash
# Render template
gtext render template.gtext

# Preview without writing
gtext render template.gtext --dry-run

# Refresh (re-render)
gtext refresh template.gtext
```

## Next Steps

Ready for advanced techniques? Continue with:

**Notebook 02: RAG & Prompt Engineering**
- Build AI/LLM prompts with dynamic context
- Create RAG pipelines
- Integrate with OpenAI, Anthropic, etc.
- Version and track prompts in git

## Resources

- 📚 [Full Protocol Reference](https://gtext.readthedocs.io/en/latest/extensions/include.html)
- 🐙 [GitHub Repository](https://github.com/genropy/gtext)
- 💬 [Community Discussions](https://github.com/genropy/gtext/discussions)

---

**Like a weaverbird, gtext weaves your dynamic content perfectly! 🪶**